In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!python3 -c "import torch; print(torch.__version__)"
!pip3 install networkx
!pip install torch-geometric

2.1.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.7 MB/s eta 0:00:00


In [3]:
#!pip3 install torch torchvision
import torch
import torch.nn as nn
import numpy as np
from numpy import inf
from tqdm import tqdm

from timeit import default_timer as timer


import networkx as nx

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# Model Architecture

In [4]:
class BaseGNNLayer(nn.Module):

    def __init__(self, input_dim, output_dim):
        super().__init__()
        lim = 1
        self.w_self = (-2 * lim * torch.rand(input_dim, output_dim) + lim).to(device)
        self.w_neigh = (-2 * lim * torch.rand(input_dim, output_dim) + lim).to(device)

    def forward(self, node_feats, adj_matrix):
        node_feats_self = torch.mm(node_feats, self.w_self)
        node_feats_neigh = torch.mm(torch.mm(adj_matrix, node_feats), self.w_neigh)

        next_node_feats = node_feats_self + \
                        node_feats_neigh
        return next_node_feats

In [5]:
threshold = 1
def act_fn(x):
    return torch.clamp(x, min=-threshold, max=threshold)

In [6]:
class BaseGNNModule(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, act_fn=act_fn):
        super().__init__()
        self.layers = nn.ModuleList([BaseGNNLayer(input_dim, hidden_dim)]).to(device)
        for i in range(num_layers - 2):
            self.layers.append(BaseGNNLayer(hidden_dim, hidden_dim).to(device))
        self.layers.append(BaseGNNLayer(hidden_dim, output_dim).to(device))
        self.act_fn = act_fn

    def forward(self, x, adj_matrix):
        ### Add randomization !!!
        x = torch.rand(x.shape).to(device)
        for layer in self.layers:
            x = self.act_fn(layer(x, adj_matrix))
        return x

In [7]:
class MLPModule(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2, act_fn=torch.tanh):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(input_dim, hidden_dim)]).to(device)
        for i in range(num_layers - 2):
            self.layers.append(nn.Linear(hidden_dim, hidden_dim).to(device))
        self.layers.append(nn.Linear(hidden_dim, output_dim).to(device))
        self.act_fn = act_fn

    def forward(self, x):
        for layer in self.layers:
            x = self.act_fn(layer(x))
        return torch.sigmoid(x)

In [8]:
def loader(A):
  n = len(A)
  for i in range(n):
    yield A[i]

# Model Evaluation

In [9]:
import torch_geometric.utils as U
import random
from torch_geometric.datasets import CitationFull
# https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.datasets.CitationFull.html#torch_geometric.datasets.CitationFull

# Cora Dataset

In [10]:
# Load Cora-ML dataset
data = CitationFull(root='', name = "Cora_ML")

# Access the dataset's graph and labels
cora_ML = data[0]
cora_ML_adj = U.to_dense_adj(cora_ML.edge_index).squeeze(0).to(device)
cora_ML_x = cora_ML.x[:,0:64].to(device)
print(cora_ML)
new_x = cora_ML.x[:,0:64]
print(new_x.shape)
print(cora_ML_adj.shape)
torch.mean(torch.sum(cora_ML_adj, 1)).item()  ### mean degree

Processing...
Done!


Data(x=[2995, 2879], edge_index=[2, 16316], y=[2995])
torch.Size([2995, 64])
torch.Size([2995, 2995])


5.447746276855469

# Simulation

In [23]:
def get_csv(graph_func, NUM_NODES, num_layers = 3, num_models=10, print_info=False, r=0.5):
  """
  graph_func(graph_dim, r) -> adj_matrix
  """
  csv = []; cora_eval = []
  d = 64; num_layers = 3;

  # Calculate graphs upfront
  A = [];
  for graph_dim in NUM_NODES:
    start_time = timer()
    for idx in range(2**5):
      A.append(graph_func(graph_dim, r))
    print(f'Graphs of size {graph_dim} generated in {timer()-start_time}')

  for mpnn_idx in range(num_models):  # There will be 10 plots.
      # Initialize random BaseGNN model with sum aggregation.
      base_gnn = BaseGNNModule(input_dim=d, hidden_dim=d, output_dim=d,
                              num_layers=num_layers, act_fn=act_fn).to(device)

      # Initialize random MLP classifier acting on final mean-pooled embedding.
      mlp = MLPModule(input_dim=d, hidden_dim=100, output_dim=1,
                  num_layers=2, act_fn=torch.tanh).to(device)

      proportions = []
      adj_loader = loader(A)

      for graph_dim in NUM_NODES:
          classifications = []

          for idx in range(2**5):
              adj_matrix = next(adj_loader).to(device)
              initial_node_feats = torch.rand(graph_dim, d).to(device)

              # Obtain final mean-pooled embedding vector over all graph_dim nodes.
              output = base_gnn(initial_node_feats, adj_matrix).mean(axis=0)

              # Apply MLP classifier to the resulting output.
              apply_classifier = mlp(output)

              # If smaller than 1/2, output 0, else output 1.
              if apply_classifier <= 0.5:
                  classifications.append(0)
              else:
                  classifications.append(1)

          # Calculate proportion of graphs classified as 1.
          classifications = np.array(classifications)
          proportions.append((classifications == 1).sum())

      csv.append([mpnn_idx, proportions])

      # Evaluate the model on Cora_ML dataset
      output = base_gnn(cora_ML_x, cora_ML_adj).mean(axis=0)
      apply_classifier = mlp(output)
      if apply_classifier <= 0.5:
        cora_eval.append([mpnn_idx, 0])
      else:
        cora_eval.append([mpnn_idx, 1])

      if print_info:
        print(csv[-1])
        print(cora_eval[-1])

  print(f"Cora predictions differ on {count_csv_scores(csv, cora_eval)} out of {num_models} models.")
  return csv, cora_eval

In [12]:
def count_csv_scores(csv, cora_eval):
  count = 0
  for i in range(len(csv)):
    score1 = csv[i][-1][-1]
    if score1 > 32/2:
      score1 = 1
    else:
      score1 = 0
    score2 = cora_eval[i][-1]
    count += int(score1 != score2)
  return count

In [13]:
# csv_ba[0][5][-1][-1]
# cora_eval[0][-1]
# print(csv_er)
# count_csv_scores(csv_er, cora_eval)

In [14]:
# adj_1 = get_ba_graph(10)
# torch.mean(torch.sum(adj_1, 1)).item()

In [15]:
# def get_internet_graph(graph_dim, r = None):
#   G = nx.random_internet_as_graph(graph_dim)
#   A = nx.adjacency_matrix(G)
#   adj_matrix = torch.tensor(A.todense()).float() #.to(device)
#   return adj_matrix

In [16]:
# def get_regular_graph(graph_dim, r = None):
#   G = nx.random_regular_graph(d=6, n=graph_dim)
#   A = nx.adjacency_matrix(G)
#   adj_matrix = torch.tensor(A.todense()).float() #.to(device)
#   return adj_matrix

In [17]:
def get_ext_ba_graph(graph_dim, r = None):
  G = nx.extended_barabasi_albert_graph(n=graph_dim, m=2, p=0.2, q=0.1)
  A = nx.adjacency_matrix(G)
  adj_matrix = torch.tensor(A.todense()).float() #.to(device)
  return adj_matrix

In [18]:
adj_1 = get_ext_ba_graph(1000)
torch.mean(torch.sum(adj_1, 1)).item()

5.111999988555908

In [19]:
import matplotlib.pyplot as plt
import pickle

In [37]:
num_models = 100

ER Graph

In [38]:
def get_er_graph(graph_dim, r):
  half_matrix = torch.bernoulli(r * (torch.triu(torch.ones(graph_dim, graph_dim)) -
                                     torch.eye(graph_dim)))
  adj_matrix = half_matrix + half_matrix.T
  return adj_matrix

NUM_NODES = [10, 50, 100, 500, 1000, 2000, 5000] #40sec
csv_er, cora_eval = get_csv(get_er_graph, NUM_NODES, num_models=200)

### 6 mistakes
### 4 mistakes
### 6 mistakes

Graphs of size 10 generated in 0.003060189999814611
Graphs of size 50 generated in 0.003885026999796537
Graphs of size 100 generated in 0.006594823999876098
Graphs of size 500 generated in 0.12474669800030824
Graphs of size 1000 generated in 0.5140862690000176
Graphs of size 2000 generated in 3.176514853000299
Graphs of size 5000 generated in 32.26770938699974
Cora predictions differ on 51 out of 200 models.


BA graph

In [39]:
### BA GRAPH
def get_ba_graph(graph_dim, r=None):
  G = nx.barabasi_albert_graph(graph_dim, 3)#, graph_dim // 2)
  A = nx.adjacency_matrix(G)
  adj_matrix = torch.tensor(A.todense()).float() #.to(device)
  return adj_matrix

NUM_NODES = [10, 50, 100, 500, 1000]
csv_ba = get_csv(get_ba_graph, NUM_NODES, num_models=200)

### No errors!!!
### 1 error
### 1 error
### 1 error

Graphs of size 10 generated in 0.0236565770001107
Graphs of size 50 generated in 0.053950079000060214
Graphs of size 100 generated in 0.08718081399956645
Graphs of size 500 generated in 0.3853006919998734
Graphs of size 1000 generated in 0.605139558999781
Cora predictions differ on 13 out of 200 models.


n/2 BA Graph

In [40]:
### BA GRAPH
def get_their_ba_graph(graph_dim, r=None):
  G = nx.barabasi_albert_graph(graph_dim, graph_dim // 2)
  A = nx.adjacency_matrix(G)
  adj_matrix = torch.tensor(A.todense()).float() #.to(device)
  return adj_matrix

NUM_NODES = [10, 50, 100, 500, 1000] #60sec
csv_ba_their = get_csv(get_their_ba_graph, NUM_NODES, num_models=200)

### 6 errors!!!
### 12 errors
### 5
### 8

Graphs of size 10 generated in 0.017364619000090897
Graphs of size 50 generated in 0.09594061200004944
Graphs of size 100 generated in 0.3332312739998997
Graphs of size 500 generated in 12.092087898000045
Graphs of size 1000 generated in 50.5101929020002
Cora predictions differ on 52 out of 200 models.


Extended BA model

In [41]:
## Extended BA model
# https://networkx.org/documentation/stable/reference/generated/networkx.generators.random_graphs.extended_barabasi_albert_graph.html#extended-barabasi-albert-graph
NUM_NODES = [10, 50, 100, 500, 1000] #60sec
csv_ext_ba, cora_csv = get_csv(get_ext_ba_graph, NUM_NODES, num_models=200)

### Tylko 2 błędy!!!!!!!! (out of 30)
### 1 error
### 0

Graphs of size 10 generated in 0.01813471000014033
Graphs of size 50 generated in 0.06888934400012658
Graphs of size 100 generated in 0.1929157640001904
Graphs of size 500 generated in 3.409352927999862
Graphs of size 1000 generated in 13.836906485000327
Cora predictions differ on 5 out of 200 models.
